In [2]:
import pandas as pd
csv_nagr = pd.read_csv('Nret_AO(2019-2025).csv')
csv_lagr = pd.read_csv('lret_AO(2019-2025).csv')
Nagr = csv_nagr['c1']
lagr = csv_lagr['c1']


Dret = (Nagr/Nagr.shift(1)/lagr) * 10000
Dret[0]=107.0

Dret

0     107.000000
1     106.963706
2     106.954772
3     105.765348
4     105.455083
5     105.703438
6     105.718445
7            NaN
8            NaN
9            NaN
10           NaN
11           NaN
12           NaN
13           NaN
14           NaN
15           NaN
16           NaN
17           NaN
18           NaN
19           NaN
20           NaN
Name: c1, dtype: float64